<a href="https://colab.research.google.com/github/tennille-bernard/Kal-Academy-Assignments/blob/main/CNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import packages**  
- in *datasets* from tensorflow, they have access to the cifar10 data (https://www.cs.toronto.edu/~kriz/cifar.html) as well as a few other datasets.  It has training and test images already set up in the dataset.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt

**Load & preprocess data: Set test images and training images**

In [ ]:
(train_images, train_labels),(test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0   #normalizing the data. This will result in (slightly) blurred images.

**Data Augmentation**, which adds variety to the images so that it can recognize more variations in the training images so that it can have a more accurate prediction.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
datagen.fit(train_images)

Old code below

In [ ]:
#class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
#plt.figure(figsize=(10,10))
#for i in range(25):
#    plt.subplot(5,5,i+1)
#    plt.xticks([])
#    plt.yticks([])
#    plt.grid(False)
#    plt.imshow(train_images[i])
#    plt.xlabel(class_names[train_labels[i][0]])
#plt.show()

**Improved CNN Model with Regularization**  
Old code commented out, updated code below

In [ ]:
#model = models.Sequential()
#model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
#model.add(layers.MaxPooling2D((2, 2)))
##added regularizer
#model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
#model.add(layers.MaxPooling2D((2, 2)))
##changed to 128
#model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
##Added another maxpool
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Flatten())  #the number of filter in the flatten layer need to match the number of filters in the last convolution layer.
##update
#model.add(layers.Dense(128, activation='relu'))
##added
#model.add(layers.Dropout(0.3))
#model.add(layers.Dense(10))

In [ ]:
#model.summary()

In the rewrite below, we created an array with all the steps as layers.  

We also added a **regularization** and increased the second **Conv2D Layer** to 128 (this indentifies more features), and added 1 more MaxPooling2D layer after.  

**Regularization**: There are 3 types:
-



In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10)
])

**Compiling the model**  
- from_logits = True --> the data here is raw values, not log values, so the setting must be true.

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

#SparseCategoriacalCrossentropy = used to calculate loss function, but this is best when using multi-class classifications for large images. This also saves your memory.

**Training the model with Augmented Data**  
by fitting the model (passing the datagen.flow with the training inputs) and creating a new variable, history.

In [ ]:
#updated
history = model.fit(datagen.flow(train_images, train_labels, batch_size=64),
                    epochs=15, validation_data=(test_images, test_labels))

**Plot Training History**  
Old code commented out, new code below

In [ ]:
#plt.plot(history.history['accuracy'], label='accuracy')
#plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
#plt.xlabel('Epoch')
#plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
#plt.legend(loc='lower right')
#test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
#print(test_acc)

In [ ]:
#import numpy as np
#from tensorflow.keras.preprocessing import image
#test_image = image.load_img('deer.jpg', target_size=(32, 32))
##updated
#test_image = image.img_to_array(test_image) / 255.0
#test_image = np.expand_dims(test_image, axis=0)


#result = model.predict(test_image)
##updated
#probabilities = tf.nn.softmax(result)
#predicted_class = np.argmax(probabilities)
#print(f"Predicted class: {class_names[predicted_class]} with confidence {probabilities.numpy()[0][predicted_class]:.2f}")

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()


Loss should decreate, accuracy should increase.

**Improved Inference Code**


In [ ]:
def predict_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(32, 32))
    img = image.img_to_array(img) / 255.0 #don't forget to normalize the data
    img = np.expand_dims(img, axis=0)
    logits = model.predict(img)
    probabilities = tf.nn.softmax(logits)  #converts large numbers to 1s and 0s
    predicted_class = np.argmax(probabilities)
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    print(f"Predicted class: {class_names[predicted_class]} with confidence {probabilities.numpy()[0][predicted_class]:.2f}")

**Example Usage**

In [ ]:
predict_image('deer.jpg')